In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, ElectraForSequenceClassification, TrainingArguments, Trainer, AutoModel, AutoModelForSequenceClassification
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:", device)



In [ ]:
df = pd.read_csv("dataset_path", sep="\t")
df.head()

In [ ]:
df.info()

### sep를 tap으로 하고 content하고 label을 분리하면 content의 끝문자열에 특수문자가 있는 경우 정상적으로 sep가 안될 수 있다. df.info를 통해 확인하고 content의 데이터수와 label의 데이터수가 다르다면 다음 코드를 실행해준다.

In [ ]:
null_idx = df[df.lable.isnull()].index

# lable 은 content의 가장 끝 문자열로 설정.
df.loc[null_idx, "lable"] = df.loc[null_idx, "content"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정.
df.loc[null_idx, "content"] = df.loc[null_idx, "content"].apply(lambda x: x[:-2])
df = df.astype({"lable":"int"})

In [ ]:
df.info()

In [ ]:
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)


In [ ]:
#데이터 중복 제거 코드 

train_data.drop_duplicates(subset=["content"], inplace= True)
test_data.drop_duplicates(subset=["content"], inplace= True)

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))

In [ ]:
MODEL_NAME = "beomi/KcELECTRA-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenized_train_sentences = tokenizer(
    list(train_data["content"]),
    return_tensors="pt",                # pytorch의 tensor 형태로 return
    max_length=128,                     # 최대 토큰길이 설정
    padding=True,                       # 제로패딩 설정
    truncation=True,                    # max_length 초과 토큰 truncate
    add_special_tokens=True,            # special token 추가
    )

# print(tokenized_train_sentences[0])
# print(tokenized_train_sentences[0].tokens)
# print(tokenized_train_sentences[0].ids)
# print(tokenized_train_sentences[0].attention_mask)


tokenized_test_sentences = tokenizer(
    list(test_data["content"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True,
    )

# print(tokenized_test_sentences[0])
# print(tokenized_test_sentences[0].tokens)
# print(tokenized_test_sentences[0].ids)
# print(tokenized_test_sentences[0].attention_mask)



In [ ]:
from Dataset import CustomDataset

train_label = train_data["lable"].values
test_label = test_data["lable"].values

train_dataset = CustomDataset(tokenized_train_sentences, train_label)
test_dataset = CustomDataset(tokenized_test_sentences, test_label)


### Discriminator만을 구조로 사용하는 ElectraForSequenceClassification (model에 Generator 아키텍쳐는 포함되지 않는다. 포함되는 경우는 자연어 생성의 경우가 있다.)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('beomi/KcELECTRA-base', num_labels=2)
model.to(device)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='./',                    # 학습결과 저장경로
    num_train_epochs=10,                # 학습 epoch 설정
    per_device_train_batch_size=8,      # train batch_size 설정
    per_device_eval_batch_size=64,      # test batch_size 설정
    logging_dir='./logs',               # 학습log 저장경로
    logging_steps=500,                  # 학습log 기록 단위
    save_total_limit=2,                 # 학습결과 저장 최대갯수
)


trainer = Trainer(
    model=model,                         # 학습하고자하는  Transformers model
    args=training_args,                  # 위에서 정의한 Training Arguments
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=test_dataset,           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가지표
)

In [ ]:
trainer.train()

# Save trained Model

In [ ]:
model.save_pretrained('model.pt')